In [ ]:
import zmq
import struct


ctx = zmq.Context.instance()

# Generate the subscriber sockets with the same settings
# as the original C++ program.
subscriber = ctx.socket(zmq.SUB)
subscriber.setsockopt(zmq.RCVHWM, 0)
subscriber.setsockopt(zmq.RCVBUF, 10*20000*1030)
subscriber.setsockopt_string(zmq.SUBSCRIBE, "")
subscriber.setsockopt(zmq.RCVTIMEO, 100)
subscriber.connect('tcp://localhost:7205')

# Also generate a similar API socket.
# api_socket = ctx.socket(zmq.REQ)
# api_socket.setsockopt(zmq.RCVTIMEO, -1)
# api_socket.connect('tcp://localhost:7210')

# This first loop seems to perform synchronization.
while True:
    try:
        msg = subscriber.recv()
    except:
        continue

    more = subscriber.getsockopt(zmq.RCVMORE)
    if not more:
        break

while True:
    # Sometimes the publisher will be interrupted, so don't
    # let that crash the entire program.
    frame_number = frame_data = events_data = None
    try:
        frame_number = struct.unpack('Q', subscriber.recv())[0]

        # We're still ignoring the frame data, but we have to
        # get it out from the data stream.
        if subscriber.getsockopt(zmq.RCVMORE):
            frame_data = subscriber.recv()

        # This is the one that stores all the spikes.
        if subscriber.getsockopt(zmq.RCVMORE):
            events_data = subscriber.recv()

    except Exception as e:
        print(e)
        continue

    events = []
    if events_data is not None:
        for i in range(0, len(events_data), 24):
            events.append(struct.unpack('8xLif',
                events_data[i:i+24]))
    if len(events) > 0:
        print(events)

del subscriber
ctx.term()